C'est marrant à quel point l'iconographie du site d'Emmanuel est soignée. Peut-on en faire une gallerie d'images?

On part de cette url : https://en-marche.fr/emmanuel-macron/le-programme 

In [4]:
from bs4 import BeautifulSoup

In [7]:
import requests

In [9]:
r = requests.get('https://en-marche.fr/emmanuel-macron/le-programme')

In [10]:
soup = BeautifulSoup(r.text, 'html.parser')

In [18]:
proposals = soup.find_all(class_='programme__proposal')

In [19]:
proposals = [p for p in proposals if 'programme__proposal--category' not in p.attrs['class']]

In [20]:
len(proposals)

36

In [22]:
p = proposals[0]

In [26]:
full_url = 'https://en-marche.fr' + p.find('a').attrs['href']
full_url

'https://en-marche.fr/emmanuel-macron/le-programme/action-publique-fonction-publique'

In [52]:
full_urls = ['https://en-marche.fr' + p.find('a').attrs['href'] for p in proposals]

In [88]:
full_urls[:10]

['https://en-marche.fr/emmanuel-macron/le-programme/action-publique-fonction-publique',
 'https://en-marche.fr/emmanuel-macron/le-programme/agriculture',
 'https://en-marche.fr/emmanuel-macron/le-programme/culture',
 'https://en-marche.fr/emmanuel-macron/le-programme/defense',
 'https://en-marche.fr/emmanuel-macron/le-programme/dependance',
 'https://en-marche.fr/emmanuel-macron/le-programme/dialogue-social',
 'https://en-marche.fr/emmanuel-macron/le-programme/education',
 'https://en-marche.fr/emmanuel-macron/le-programme/%C3%A9galit%C3%A9-hommes-et-femmes',
 'https://en-marche.fr/emmanuel-macron/le-programme/emploi-ch%C3%B4mage-securites-professionnelles',
 'https://en-marche.fr/emmanuel-macron/le-programme/enseignement-superieur-recherche']

In [99]:
r = requests.get(full_url)
soup = BeautifulSoup(r.text, 'html.parser')

In [101]:
figure_tag = soup.find('figure', class_='fullscreen')
figure_tag

<figure class="fullscreen">
<img alt="01-fonction-publique-hospital-sante-emmanuel-macron-en-marche" src="/assets/images/01-fonction-publique-hospital-sante-emmanuel-macron-en-marche?q=70&amp;cache=e7d04db7e6ec8a188aee&amp;fm=pjpg&amp;s=97b9c84c57c417dcef72c4919e6f2625" title="Action publique / Fonction publique"/>
</figure>

On peut maintenant extraire le lien vers l'image.

In [102]:
src_url = 'https://en-marche.fr' + figure_tag('img')[0].attrs['src']
src_url

'https://en-marche.fr/assets/images/01-fonction-publique-hospital-sante-emmanuel-macron-en-marche?q=70&cache=e7d04db7e6ec8a188aee&fm=pjpg&s=97b9c84c57c417dcef72c4919e6f2625'

On peut afficher ceci dans le notebook.

In [49]:
from IPython.display import Image

In [50]:
Image(url=src_url)

In [103]:
def extract_img_src(url):
    "Extracts image src url from linked page."
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    figure_tag = soup.find('figure', class_='fullscreen')
    if figure_tag is not None and figure_tag('img') is not None:
        src_url = 'https://en-marche.fr' + figure_tag('img')[0].attrs['src']
        return src_url
    else:
        print("no image for url: {}".format(url))
        return None

On peut répeter ce processus et faire une gallerie avec toutes ces images.

In [110]:
image_srcs = [extract_img_src(url) for url in full_urls]

no image for url: https://en-marche.fr/emmanuel-macron/le-programme/familles-et-societe
no image for url: https://en-marche.fr/emmanuel-macron/le-programme/handicap
no image for url: https://en-marche.fr/emmanuel-macron/le-programme/immigration-et-asile
no image for url: https://en-marche.fr/emmanuel-macron/le-programme/justice
no image for url: https://en-marche.fr/emmanuel-macron/le-programme/pauvrete


In [111]:
image_srcs = [_ for _ in image_srcs if _ is not None]

In [112]:
header = """<!doctype html>
<html lang="fr">
<head>
  <meta charset="utf-8">
  <title>Gallerie des photos du site d'Emmanuel Macron</title>
  <style>
  img {width: 100%;}
  </style>
</head>"""

In [113]:
image_srcs[0]

'https://en-marche.fr/assets/images/01-fonction-publique-hospital-sante-emmanuel-macron-en-marche?q=70&cache=e7d04db7e6ec8a188aee&fm=pjpg&s=97b9c84c57c417dcef72c4919e6f2625'

In [114]:
def format_as_img_tag(src):
    return "<img src={} />".format(src)

In [115]:
format_as_img_tag(image_srcs[2])

'<img src=https://en-marche.fr/assets/images/04-culture-musee-exposition-guadeloupe-emmanuel-macron?q=70&cache=0f9e2f1675c10ef5c67b&fm=pjpg&s=92575e4c67cd6a07095acfd08652efb6 />'

In [116]:
with open('gallerie_macron.html', 'w') as f:
    body = """<body>
{0}
</body>""".format("\n".join(format_as_img_tag(url) for url in image_srcs))
    html = header + body + "</html>"
    f.write(html)

Ce sont des belles photos...